# Introduction/Business Problem
In the capstone project for IBM certificate course, the aim is to analyse the accident "severity" whether in terms of fatality or traffic or accidents or any other type of bad impact. All records were provided by SPD and recorded by Traffic Records and provided to me by Coursera. This dataset covers a period of 2004 to present. It contains information such as severity code, address type, weather, road condition, speeding, etc.

The audience will be road users who will benefit from information that will possibly help reduce traffic collision/accidents in the future and also assist with travel plans. This model can alert/assist road users know when the road is safe or bad so they can plan accordingly

# Data Understanding

There are 194,673 observations and 38 attributes in the dataset. The aim is to identify the severity of an accident and the possible factors that can lead to one. From the dataset, the SEVERITYCODE attribute will be used as the target variable (Y). The two types of severity are 1-prop damage and 2-injury
Feature Engineering process will be applied to ascertain the attributes that impact the severity of an accident. Missing value will either be replaced or removed from the dataset as well.

The columns that I will be using for the analysis are as follows:

- COLLISIONTYPE, which describes the type of crash, 
- WEATHER, description of the weather conditions during the time of the collision
- ROADCOND, condition of the road during the collision.
- LIGHTCOND, light conditions during the collision, 
- INATTENTIONIND, whether or not collision was due to inattention. (Y/N) 
- UNDERINFL, whether or not a driver involved was under the influence of drugs or alcohol.
- SEVERITYCODE, corresponds to the severity of the collision and assigns a crash a value of 1, which means property damage collison, and 2, indicating injury